In [0]:
#mounting drive on colab
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd

In [0]:
#importing the confirmed global cases file from the JHU repository
covid_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [0]:
covid_confirmed.head(5)

In [0]:
#importing the deaths global file from the JHU repository
covid_deaths=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [0]:
covid_deaths.head()

In [0]:
#importing the recovered global cases file from the JHU repository
covid_recovered=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [0]:
covid_recovered.head()

In [0]:
#converting the time series data into  
covid_confirmed=covid_confirmed.melt(id_vars=['Province/State','Country/Region','Lat','Long'])

In [0]:
covid_confirmed.head()

In [0]:
#renaming the 'variable' and 'value' to suitable variables
covid_confirmed=covid_confirmed.rename(columns={'variable':'Date','value':'Confirmed'})

In [0]:
#similarly applying it to other dataframes
covid_deaths=covid_deaths.melt(id_vars=['Province/State','Country/Region','Lat','Long'])
covid_deaths=covid_deaths.rename(columns={'variable':'Date','value':'Deaths'})

In [0]:
covid_recovered=covid_recovered.melt(id_vars=['Province/State','Country/Region','Lat','Long'])
covid_recovered=covid_recovered.rename(columns={'variable':'Date','value':'Recovered'})

In [0]:
covid_deaths.head()

In [0]:
#combining the three dataframes into one
combined_df=[covid_confirmed,covid_deaths,covid_recovered]
combined_df=[df.set_index(['Province/State','Country/Region','Lat','Long','Date']) for df in combined_df]
combined_df=combined_df[0].join(combined_df[1:])

In [0]:
combined_df.head()

In [0]:
#resetting the index
combined_df=combined_df.reset_index()

In [0]:
combined_df.head()

In [0]:
#updating the default data types
combined_df[['Lat','Long','Confirmed','Deaths','Recovered']]=combined_df[['Lat','Long','Confirmed','Deaths','Recovered']].apply(pd.to_numeric)
combined_df[['Date']]=combined_df[['Date']].apply(pd.to_datetime)

In [0]:
#renaming columns
combined_df=combined_df.rename(columns={'Province/State':'State','Country/Region':'Country'})

In [0]:
combined_df.head()

In [0]:
#missing values in the dataframe
combined_df.isnull().sum()     

In [0]:
combined_df.count()

In [0]:
#many countries are mapped at Country level rather than State(China)
combined_df.set_index('Country').filter(like='India',axis=0)

In [0]:
combined_df[combined_df['State'].isnull()]['Country'].value_counts()

In [0]:
#to fill up the null state values with capitals
capital_df=pd.read_csv('https://raw.githubusercontent.com/icyrockcom/country-capitals/master/data/country-list.csv')

In [0]:
capital_df.head(50)

In [0]:
#correcting the country name
capital_df['country'].replace({"United Kingdom; England":"United Kingdom"},inplace=True)

In [0]:
capital_df['country']=capital_df.country.str.lower()

In [0]:
capital_df.head()

In [0]:
#easy to look up the capital value
capital_df=capital_df.set_index('country')

In [0]:
capital_df.head()

In [0]:
#function to replace missing states with capitals
def lookup_capital(country):
  try:
    capital=capital_df.loc[country.lower()][0]
  except:
    capital="TBF"
  return capital

In [0]:
combined_df_c=combined_df.copy()

In [0]:
capital_df.loc['iran'][0]

In [0]:
combined_df_c[combined_df_c['Recovered'].isnull()]['Country']

In [0]:
#replacing null values with TBF
l={'State':"TBF"}
combined_df_c.fillna(l,inplace=True)

In [0]:
#applying the lookup function
combined_df_c['State']=combined_df_c.apply(lambda x: lookup_capital(x['Country']) if (x['State']== "TBF") else x['State'], axis=1)

In [0]:
combined_df_c.head()

In [0]:
#checking for the countries still having the missing states
combined_df_c[combined_df_c.State=="TBF"]['Country'].value_counts()

In [0]:
#replacing exising country names with the correct names to find the appropriate capitals
combined_df_c["Country"].replace({"Czechia": "Czech Republic", "Holy See": "Vatican City", "Korea, South": "South Korea","Burma":"Myanmar","South Sudan":"Sudan","North Macedonia":"Macedonia"}, inplace=True)


In [0]:
#capital_df.loc["ivory coast"][0]

In [0]:
#applying the lookup function again
combined_df_c['State']=combined_df_c.apply(lambda x: lookup_capital(x['Country']) if (x['State']== "TBF") else x['State'], axis=1)

In [0]:
combined_df_c[combined_df_c.State=="TBF"]['Country'].value_counts()

In [0]:
#ignoring the values with missing states after required processing
combined_df_c['State']=combined_df_c.apply(lambda x: x['Country'] if (x['State']== "TBF") else x['State'], axis=1)

In [0]:
combined_df_c[combined_df_c.State=="TBF"]['Country'].value_counts()

In [0]:
#combined_df_c.to_csv('covid_dataset')

In [0]:
#not imputing the few missing values in revovered column as the current focus is not to create a model
combined_df_c[combined_df_c.Recovered.isnull()]

In [0]:
#s={"Recovered":0}
#combined_df_c.fillna(s,inplace=True)

In [0]:
#combined_df_c.isnull().sum()

In [0]:
#combined_df_c.to_csv("/content/drive/My Drive/covid19_data_new.csv")

In [0]:
#the revised JHU global repository has US mapped at country level
combined_df_c[combined_df_c.Country=="US"]

In [0]:
#integrating the separate US data sets(confirmed and deaths) with our transformed global dataset
conf_us=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

In [0]:
conf_us.head()

In [0]:
#dropping variable which are not required
conf_us.drop(['UID','iso2','iso3','code3', 'FIPS','Admin2','Combined_Key'],inplace=True,axis=1)

In [0]:
conf_us.head()

In [0]:
conf_us=conf_us.melt(id_vars=['Province_State','Country_Region','Lat','Long_'])

In [0]:
conf_us.head()

In [0]:
conf_us.shape

In [0]:
#renaming the columns 
conf_us.rename(columns={'Long_':'Long',"Province_State":"State","Country_Region":"Country","variable":"Date","value":"Confirmed"},inplace=True)

In [0]:
conf_us.head()

In [0]:
#repeating the previous steps for US deaths data set
deaths_us=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

In [0]:
deaths_us.head()

In [0]:
deaths_us.drop(['UID','iso2','iso3','code3', 'FIPS','Admin2','Combined_Key',"Population"],inplace=True,axis=1)

In [0]:
#using melt function to convert the data into suitable format for analysis
deaths_us=deaths_us.melt(id_vars=['Province_State','Country_Region','Lat','Long_'])

In [0]:
deaths_us.rename(columns={'Long_':'Long',"Province_State":"State","Country_Region":"Country","variable":"Date","value":"Deaths"},inplace=True)

In [0]:
deaths_us.head()

In [0]:
#combining the US confirmed and US deaths into one
combined_df_us=[conf_us,deaths_us]
combined_df_us=[df.set_index(['State','Country','Lat','Long','Date']) for df in combined_df_us]
combined_df_us=pd.concat([combined_df_us[0],combined_df_us[1]],axis=1)

In [0]:
combined_df_us=combined_df_us.reset_index()


In [0]:
combined_df_us.head()

In [0]:
combined_df_us.tail()

In [0]:
#adding a 'Recovered' column 
import numpy as np
combined_df_us['Recovered']= np.nan

In [0]:
combined_df_us.head()

In [0]:
combined_df_us.shape

In [0]:
combined_df_c.tail()


In [0]:
#concatenating our previous Global dataset with the transormed US dataset
combined_final=pd.concat([combined_df_c,combined_df_us],ignore_index=True)

In [0]:
combined_df_c.head()

In [0]:
combined_final[['Date']]=combined_final[['Date']].apply(pd.to_datetime)

In [0]:
combined_final.dtypes

In [0]:
combined_final.tail()

In [0]:
combined_final.head()

In [0]:
#pushing the final cleaned and transformed data to drive
combined_final.to_csv('/content/drive/My Drive/covid_final.csv')